# 06 — Functions & Scope

Goal: Turn repeated logic into reusable building blocks and understand where variables “live”.

This is the bridge from:
> “I can write code”  
to  
> “I can structure a training loop, preprocessing pipeline, or loss function cleanly.”


## 1. What Is a Function?

A **function** is a named block of code that:

- takes **inputs** (parameters/arguments)
- does some work
- optionally **returns** a value

Syntax:

```python
def name(parameters):
    # body
    return something
``` 

In [ ]:
def greet():
    print("Hello from a function!")

greet()

def square(x):
    return x * x

print("square(3) =", square(3))
print("square(10) =", square(10))

## 2. Parameters vs Arguments

- **Parameters** are the names in the function definition:

```python
def add(a, b):  # a, b = parameters
    return a + b
```
- Arguments are the actual values you pass in:

```python
add(2, 3)       # 2, 3 = arguments

``` 

In [ ]:
def add(a, b):
    return a + b

result = add(2, 3)
print("add(2, 3) =", result)

result = add(10, -4)
print("add(10, -4) =", result)

## 3. Default Arguments

You can give parameters default values:

```python
def log(message, level="INFO"):
    print(f"[{level}] {message}")
``` 
If the caller doesn’t provide level, "INFO" is used.

In [ ]:
def log(message, level="INFO"):
    print(f"[{level}] {message}")

log("Starting training...")
log("NaN detected in loss!", level="WARN")

## 4. Keyword Arguments

You can pass arguments by **name**, not just by position.

```python
def scale(x, factor=1.0, offset=0.0):
    return x * factor + offset

scale(10, factor=0.1, offset=1.0)
``` 
This makes code more readable and less bug-prone.

In [ ]:
def scale(x, factor=1.0, offset=0.0):
    return x * factor + offset

print(scale(10))                           # default factor=1, offset=0
print(scale(10, factor=0.1))
print(scale(10, factor=0.1, offset=1.0))
print(scale(x=10, offset=5, factor=2))

## 5. Scope — Where Variables "Live"

Two main ideas:

- **Local variables**: defined inside a function → only exist inside that function.
- **Global variables**: defined at the top level of a module/notebook.

Python looks up variables in this order:
> local → enclosing → global → builtins

For now, think **“inside the function vs outside”**.


In [ ]:
x = 10  # global

def show_x():
    x = 5  # local
    print("Inside function, x =", x)

show_x()
print("Outside function, x =", x)


### Functions Can *Read* Globals

If you don't assign to a name inside the function, it will use the global value.


In [ ]:
learning_rate = 0.01

def print_lr():
    print("Current learning rate:", learning_rate)

print_lr()
learning_rate = 0.001
print_lr()


### Modifying Globals Is Dangerous

You *can* modify a global using `global`, but it's usually a bad idea:

```python
count = 0

def increment():
    global count
    count += 1
``` 
This makes code harder to reason about. <br>
In ML code, prefer returning values or updating explicit objects instead of mutating globals.

In [ ]:
count = 0

def increment():
    global count
    count += 1

increment()
increment()
print("count =", count)

## 6. Pure vs Impure Functions

A **pure function**:
- output depends *only* on its inputs
- no side effects (doesn’t modify external state)

Example:

```python
def relu(x):
    return max(0, x)
``` 
An impure function:

- touches or changes external state (globals, files, etc.)

- or depends on things like time, randomness, etc.

In ML, loss functions and activation functions are usually pure.
<br>Logging, updating parameters, writing to disk are impure.

In [ ]:
# Pure-ish
def mean(values):
    return sum(values) / len(values)

print("mean([1,2,3]) =", mean([1, 2, 3]))

# Impure: prints and mutates external list
history = []

def log_loss(loss):
    print("loss:", loss)
    history.append(loss)

log_loss(0.9)
log_loss(0.7)
print("history:", history)

## 7. Important: Mutable Default Arguments

This is a subtle but important Python quirk:

```python
def build_list(x, lst=[]):
    lst.append(x)
    return lst
```
Looks harmless, but the same list is reused across calls:
```python
build_list(1)  # [1]
build_list(2)  # [1, 2]
build_list(3)  # [1, 2, 3]
```

In [ ]:
def bad_build_list(x, lst=[]):
    lst.append(x)
    return lst

print("First call:", bad_build_list(1))
print("Second call:", bad_build_list(2))
print("Third call:", bad_build_list(3))

def good_build_list(x, lst=None):
    if lst is None:
        lst = []
    lst.append(x)
    return lst

print("Good 1:", good_build_list(1))
print("Good 2:", good_build_list(2))
print("Good 3 with existing [10]:", good_build_list(3, [10]))

## 8. ML-Flavoured Function Examples

### Loss function

```python
def mse_loss(y_true, y_pred):
    # mean squared error
    total = 0.0
    for t, p in zip(y_true, y_pred):
        diff = p - t
        total += diff * diff
    return total / len(y_true)
```
Learning rate schedule
```python
def step_lr(initial_lr, epoch, drop_every=10, factor=0.1):
    # drops LR by factor every `drop_every` epochs
    drops = epoch // drop_every
    return initial_lr * (factor ** drops)

```

# 06 — Exercises (Functions & Scope)

### Exercise 1 — Simple Function

Write a function:

```python
def square(x):
    ...
```
that returns **x** squared.
<br>Test it with **2, 5, -3**.

In [ ]:
# Excercise 1
def square(x):
    ...

print(square(2))
print(square(5))
print(square(-3))

Exercise 2 — Mean of a List

Write a function:
```python
def mean(values):
    ...
```
that returns the average of a list of numbers.
Handle the edge case of an empty list by raising a ValueError.

In [ ]:
# Excercise 2
def mean(values):
    ...

lst = [1,2,3,4,5,5,6,7,4,3,8,9,7,5,]

Exercise 3 — Clipped ReLU

Define:
```python
def clipped_relu(x, max_value):
```
Returns:

- 0 if x < 0

- x if 0 <= x <= max_value

- max_value if x > max_value

Test with a few values.

In [ ]:
# Excercise 3
def clipped_relu(x, max_value):
    ...

# Pick your own values
x = 0 # input value
max_value = 10 # limiter

Exercise 4 — Safe Build List

Re-implement safe build_list:

```python
def build_list(x, lst=None):
    ...
```

- If lst is None, start a new list.

- Otherwise, append to the given list.

Demonstrate that repeated calls without providing lst do not share state.

In [ ]:
def build_list(x, lst=None):
    ...

Exercise 5 — Simple LR Schedule

Implement:

```python
def step_lr(initial_lr, epoch, drop_every=5, factor=0.5):
    ...

```

Rules:

- For every full **drop_every** epochs, multiply lr by **factor**.

- Example: **initial_lr=0.1, epoch=0..10** should give:

    - epoch 0–4: 0.1

    - epoch 5–9: 0.05

    - epoch 10–14: 0.025 (etc.)

In [ ]:
# Excercise 5
def step_lr(initial_lr, epoch, drop_every=5, factor=0.5):
    ...

Exercise 6 — Scope Check

Predict what this prints before running:

```python
x = 5

def foo():
    x = 10
    print("inside foo:", x)

def bar():
    print("inside bar:", x)

foo()
bar()
print("outside:", x)

```

Then run it and compare.

In [ ]:
# # Excercise 6
# x = 5

# def foo():
#     x = 10
#     print("inside foo:", x)

# def bar():
#     print("inside bar:", x)

# foo()
# bar()
# print("outside:", x)